In [193]:
#import library
import requests
import pandas as pd
import numpy as np
import pandas.util.testing as tm

In [98]:
# get data from endpoint 
r= requests.get('https://www.fantasynutmeg.com/api/history/season')
df = pd.DataFrame(r.json())
print(df)

         0
0  2016-17
1  2017-18
2  2018-19
3  2019-20
4  2020-21
5  2021-22
6  2022-23


In [99]:
# get data from endpoint
r= requests.get('https://www.fantasynutmeg.com/api/history/season/2016-17')
json = r.json()

In [194]:
# get data from endpoint
r= requests.get('https://www.fantasynutmeg.com/api/history/season/2016-17')
test = r.json()

In [195]:
test.keys()

dict_keys(['dd_agg_fixture', 'dd_agg_player', 'dd_hauls', 'history'])

In [100]:
json.keys()

dict_keys(['dd_agg_fixture', 'dd_agg_player', 'dd_hauls', 'history'])

In [101]:
dd_agg_df = pd.DataFrame(json['dd_agg_fixture'])
dd_hauls16_df = pd.DataFrame(json['dd_hauls'])
dd_agg_pl_df = pd.DataFrame(json['dd_agg_player'])
hist16_df = pd.DataFrame(json['history'])

In [102]:
dd_agg_df.head()

,count,fixture
0,22,HUL(H)
1,22,WAT(H)
2,19,LEI(H)
3,19,SWA(A)
4,18,SUN(A)


In [103]:
dd_agg_df.columns

Index(['count', 'fixture'], dtype='object')

In [104]:
hist16_df.columns

Index(['assists', 'bonus', 'bps', 'chance_of_playing_next_round',
       'chance_of_playing_this_round', 'clean_sheets', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'creativity', 'dreamteam_count', 'ea_index',
       'element_type', 'ep_next', 'ep_this', 'event_points', 'first_name',
       'form', 'goals_conceded', 'goals_scored', 'ict_index', 'id',
       'in_dreamteam', 'influence', 'loaned_in', 'loaned_out', 'loans_in',
       'loans_out', 'minutes', 'news', 'now_cost', 'own_goals',
       'penalties_missed', 'penalties_saved', 'photo', 'points_per_game',
       'position', 'red_cards', 'saves', 'second_name', 'selected_by_percent',
       'special', 'squad_number', 'status', 'team', 'team_code', 'team_name',
       'threat', 'total_points', 'transfers_in', 'transfers_in_event',
       'transfers_out', 'transfers_out_event', 'value_form', 'value_season',
       'web_name', 'yellow_cards'],
      dtype='object')

In [105]:
hist16_df.shape

(683, 59)

In [106]:
dd_agg_pl_df.head()

,count,player
0,2,Defour
1,7,Coutinho
2,4,Lallana
3,4,Milner
4,5,Daniels


In [107]:
dd_agg_pl_df.columns

Index(['count', 'player'], dtype='object')

In [108]:
dd_agg_pl_df.shape

(206, 2)

In [109]:
dd_hauls16_df.head(400)

,fixture,player,points,position,round
0,HUL(H),Defour,11,MID,4
1,HUL(H),Coutinho,14,MID,6
2,HUL(H),Lallana,15,MID,6
3,HUL(H),Milner,14,MID,6
4,HUL(H),Daniels,13,DEF,8
...,...,...,...,...,...
395,LIV(H),McArthur,15,MID,10
396,LIV(H),Fraser,15,MID,14
397,LIV(H),Defoe,13,FWD,20
398,LIV(H),Ranocchia,12,DEF,24


In [110]:
dd_hauls16_df['year'] = dd_hauls16_df.apply(lambda x: "2016-17", axis=1)

In [111]:
teams=dict(zip(hist16_df.web_name, hist16_df.team_name))
dd_hauls16_df['club_name'] = dd_hauls16_df['player'].map(teams)
dd_hauls16_df.head()

,fixture,player,points,position,round,year,club_name
0,HUL(H),Defour,11,MID,4,2016-17,BUR
1,HUL(H),Coutinho,14,MID,6,2016-17,LIV
2,HUL(H),Lallana,15,MID,6,2016-17,LIV
3,HUL(H),Milner,14,MID,6,2016-17,LIV
4,HUL(H),Daniels,13,DEF,8,2016-17,BOU


In [112]:
dd_hauls16_df.shape

(469, 7)

In [113]:
dd_hauls16_df.isnull().any()

fixture      False
player       False
points       False
position     False
round        False
year         False
club_name    False
dtype: bool

In [114]:
# get data from endpoint
r= requests.get('https://www.fantasynutmeg.com/api/history/season/2017-18')
json = r.json()
json.keys()

dict_keys(['dd_agg_fixture', 'dd_agg_player', 'dd_hauls', 'history'])

In [115]:
dd_hauls17_df = pd.DataFrame(json['dd_hauls'])
hist17_df = pd.DataFrame(json['history'])
dd_hauls17_df['year'] = dd_hauls17_df.apply(lambda x: "2017-18", axis=1)
dd_hauls17_df.head()

,fixture,player,points,position,round,year
0,BHA(H),Maguire,15,DEF,2,2017-18
1,BHA(H),Mahrez,11,MID,2,2017-18
2,BHA(H),Surman,10,MID,5,2017-18
3,BHA(H),Monreal,15,DEF,7,2017-18
4,BHA(H),Bellerín,11,DEF,7,2017-18


In [116]:
dd_hauls17_df.shape

(431, 6)

In [117]:
teams=dict(zip(hist17_df.web_name, hist17_df.team_name))
dd_hauls17_df['club_name'] = dd_hauls17_df['player'].map(teams)
dd_hauls17_df.head()

,fixture,player,points,position,round,year,club_name
0,BHA(H),Maguire,15,DEF,2,2017-18,LEI
1,BHA(H),Mahrez,11,MID,2,2017-18,LEI
2,BHA(H),Surman,10,MID,5,2017-18,BOU
3,BHA(H),Monreal,15,DEF,7,2017-18,ARS
4,BHA(H),Bellerín,11,DEF,7,2017-18,ARS


In [118]:
# get data from endpoint
r= requests.get('https://www.fantasynutmeg.com/api/history/season/2018-19')
json = r.json()
json.keys()

dict_keys(['dd_agg_fixture', 'dd_agg_player', 'dd_hauls', 'history'])

In [119]:
dd_hauls18_df = pd.DataFrame(json['dd_hauls'])
hist18_df = pd.DataFrame(json['history'])
dd_hauls18_df['year'] = dd_hauls18_df.apply(lambda x: "2018-19", axis=1)
dd_hauls18_df.head()

,fixture,player,points,position,round,year
0,HUD(H),Agüero,20,FWD,2,2018-19
1,HUD(H),Vardy,11,FWD,6,2018-19
2,HUD(H),Iheanacho,12,FWD,6,2018-19
3,HUD(H),Kiko Femenía,12,DEF,10,2018-19
4,HUD(H),Pereyra,11,MID,10,2018-19


In [120]:
dd_hauls18_df.shape

(460, 6)

In [121]:
teams=dict(zip(hist18_df.web_name, hist18_df.team_name))
dd_hauls18_df['club_name'] = dd_hauls18_df['player'].map(teams)
dd_hauls18_df.head()

,fixture,player,points,position,round,year,club_name
0,HUD(H),Agüero,20,FWD,2,2018-19,MCI
1,HUD(H),Vardy,11,FWD,6,2018-19,LEI
2,HUD(H),Iheanacho,12,FWD,6,2018-19,LEI
3,HUD(H),Kiko Femenía,12,DEF,10,2018-19,WAT
4,HUD(H),Pereyra,11,MID,10,2018-19,WAT


In [122]:
# get data from endpoint
r= requests.get('https://www.fantasynutmeg.com/api/history/season/2019-20')
json = r.json()
json.keys()

dict_keys(['dd_agg_fixture', 'dd_agg_player', 'dd_hauls', 'history'])

In [123]:
dd_hauls19_df = pd.DataFrame(json['dd_hauls'])
hist19_df = pd.DataFrame(json['history'])
dd_hauls19_df['year'] = dd_hauls19_df.apply(lambda x: "2019-20", axis=1)
dd_hauls19_df.head()

,fixture,player,points,position,round,year
0,WAT(H),Digne,11,DEF,2,2019-20
1,WAT(H),Bernard,11,MID,2,2019-20
2,WAT(H),Otamendi,15,DEF,6,2019-20
3,WAT(H),De Bruyne,17,MID,6,2019-20
4,WAT(H),Mahrez,15,MID,6,2019-20


In [124]:
teams=dict(zip(hist19_df.web_name, hist19_df.team_name))
dd_hauls19_df['club_name'] = dd_hauls19_df['player'].map(teams)
dd_hauls19_df.head()

,fixture,player,points,position,round,year,club_name
0,WAT(H),Digne,11,DEF,2,2019-20,EVE
1,WAT(H),Bernard,11,MID,2,2019-20,EVE
2,WAT(H),Otamendi,15,DEF,6,2019-20,MCI
3,WAT(H),De Bruyne,17,MID,6,2019-20,MCI
4,WAT(H),Mahrez,15,MID,6,2019-20,MCI


In [125]:
dd_hauls20_df = pd.DataFrame(json['dd_hauls'])
hist20_df = pd.DataFrame(json['history'])
dd_hauls20_df['year'] = dd_hauls20_df.apply(lambda x: "2020-21", axis=1)
dd_hauls20_df.head()

,fixture,player,points,position,round,year
0,WAT(H),Digne,11,DEF,2,2020-21
1,WAT(H),Bernard,11,MID,2,2020-21
2,WAT(H),Otamendi,15,DEF,6,2020-21
3,WAT(H),De Bruyne,17,MID,6,2020-21
4,WAT(H),Mahrez,15,MID,6,2020-21


In [126]:
dd_hauls20_df.shape

(435, 6)

In [127]:
teams=dict(zip(hist20_df.web_name, hist20_df.team_name))
dd_hauls20_df['club_name'] = dd_hauls20_df['player'].map(teams)
dd_hauls20_df.head()

,fixture,player,points,position,round,year,club_name
0,WAT(H),Digne,11,DEF,2,2020-21,EVE
1,WAT(H),Bernard,11,MID,2,2020-21,EVE
2,WAT(H),Otamendi,15,DEF,6,2020-21,MCI
3,WAT(H),De Bruyne,17,MID,6,2020-21,MCI
4,WAT(H),Mahrez,15,MID,6,2020-21,MCI


In [128]:
# get data from endpoint
r= requests.get('https://www.fantasynutmeg.com/api/history/season/2021-22')
json = r.json()
json.keys()

dict_keys(['dd_agg_fixture', 'dd_agg_player', 'dd_hauls', 'history'])

In [129]:
dd_hauls21_df = pd.DataFrame(json['dd_hauls'])
hist21_df = pd.DataFrame(json['history'])
dd_hauls21_df['year'] = dd_hauls21_df.apply(lambda x: "2021-22", axis=1)
dd_hauls21_df.head()

,fixture,player,points,position,round,year
0,NOR(A),Salah,17,MID,1,2021-22
1,NOR(A),Vardy,12,FWD,3,2021-22
2,NOR(A),Sarr,15,MID,5,2021-22
3,NOR(A),King,10,FWD,5,2021-22
4,NOR(A),Sá,10,GKP,13,2021-22


In [130]:
dd_hauls21_df.shape

(472, 6)

In [131]:
teams=dict(zip(hist21_df.web_name, hist21_df.team_name))
dd_hauls21_df['club_name'] = dd_hauls21_df['player'].map(teams)
dd_hauls21_df.head()

,fixture,player,points,position,round,year,club_name
0,NOR(A),Salah,17,MID,1,2021-22,LIV
1,NOR(A),Vardy,12,FWD,3,2021-22,LEI
2,NOR(A),Sarr,15,MID,5,2021-22,CHE
3,NOR(A),King,10,FWD,5,2021-22,WAT
4,NOR(A),Sá,10,GKP,13,2021-22,WOL


In [132]:
# get data from endpoint
r= requests.get('https://www.fantasynutmeg.com/api/history/season/2022-23')
json = r.json()
json.keys()

dict_keys(['dd_agg_fixture', 'dd_agg_player', 'dd_hauls', 'history'])

In [133]:
dd_hauls22_df = pd.DataFrame(json['dd_hauls'])
hist22_df = pd.DataFrame(json['history'])
dd_hauls22_df['year'] = dd_hauls22_df.apply(lambda x: "2022-23", axis=1)
dd_hauls22_df.head()

,fixture,player,points,position,round,year
0,AVL(H),Lerma,10,MID,1,2022-23
1,AVL(H),Kelly,12,DEF,1,2022-23
2,AVL(H),Zaha,12,MID,3,2022-23
3,LIV(H),Mitrović,13,FWD,1,2022-23
4,LIV(H),Rashford,10,MID,3,2022-23


In [134]:
dd_hauls22_df.shape

(38, 6)

In [135]:
teams=dict(zip(hist22_df.web_name, hist22_df.team_name))
dd_hauls22_df['club_name'] = dd_hauls22_df['player'].map(teams)
dd_hauls22_df.head()

,fixture,player,points,position,round,year,club_name
0,AVL(H),Lerma,10,MID,1,2022-23,BOU
1,AVL(H),Kelly,12,DEF,1,2022-23,BOU
2,AVL(H),Zaha,12,MID,3,2022-23,CRY
3,LIV(H),Mitrović,13,FWD,1,2022-23,FUL
4,LIV(H),Rashford,10,MID,3,2022-23,MUN


In [136]:
dfs = [dd_hauls16_df, dd_hauls17_df, dd_hauls18_df, dd_hauls19_df, dd_hauls20_df, dd_hauls21_df]

hauls_df = pd.concat(dfs, axis=0, ignore_index=True)
hauls_df.shape

(2702, 7)

In [137]:
hauls_df.tail()

,fixture,player,points,position,round,year,club_name
2697,MCI(H),Lloris,10,GKP,1,2021-22,TOT
2698,MCI(H),Son,10,MID,1,2021-22,TOT
2699,MCI(H),Salah,13,MID,7,2021-22,LIV
2700,MCI(H),Bowen,14,MID,37,2021-22,WHU
2701,LIV(A),Shelvey,10,MID,17,2021-22,NEW


In [138]:
#Merge "round" and "year" into a column
hauls_df['aclub'] = hauls_df['fixture'].str.slice(0, 3)
dd_hauls22_df['aclub'] = dd_hauls22_df['fixture'].str.slice(0, 3)
hauls_df['GW'] = hauls_df['round'].map(str) + '-' + hauls_df['year'].map(str)
dd_hauls22_df['GW'] = dd_hauls22_df['round'].map(str) + '-' + dd_hauls22_df['year'].map(str)
hauls_df['player-opp'] = hauls_df['player'].map(str) + '-' + hauls_df['aclub'].map(str)
dd_hauls22_df['player-opp'] = dd_hauls22_df['player'].map(str) + '-' + dd_hauls22_df['aclub'].map(str)

#Create a column for Home and Away
a='(H)'

hauls_df['Ground'] = hauls_df.fixture.apply(lambda x: "Home" if a in x else "Away")
dd_hauls22_df['Ground'] = dd_hauls22_df.fixture.apply(lambda x: "Home" if a in x else "Away")

# 

In [139]:
hauls_df.tail()

,fixture,player,points,position,round,year,club_name,aclub,GW,player-opp,Ground
2697,MCI(H),Lloris,10,GKP,1,2021-22,TOT,MCI,1-2021-22,Lloris-MCI,Home
2698,MCI(H),Son,10,MID,1,2021-22,TOT,MCI,1-2021-22,Son-MCI,Home
2699,MCI(H),Salah,13,MID,7,2021-22,LIV,MCI,7-2021-22,Salah-MCI,Home
2700,MCI(H),Bowen,14,MID,37,2021-22,WHU,MCI,37-2021-22,Bowen-MCI,Home
2701,LIV(A),Shelvey,10,MID,17,2021-22,NEW,LIV,17-2021-22,Shelvey-LIV,Away


In [140]:
dd_hauls22_df.head(75)

,fixture,player,points,position,round,year,club_name,aclub,GW,player-opp,Ground
0,AVL(H),Lerma,10,MID,1,2022-23,BOU,AVL,1-2022-23,Lerma-AVL,Home
1,AVL(H),Kelly,12,DEF,1,2022-23,BOU,AVL,1-2022-23,Kelly-AVL,Home
2,AVL(H),Zaha,12,MID,3,2022-23,CRY,AVL,3-2022-23,Zaha-AVL,Home
3,LIV(H),Mitrović,13,FWD,1,2022-23,FUL,LIV,1-2022-23,Mitrović-LIV,Home
4,LIV(H),Rashford,10,MID,3,2022-23,MUN,LIV,3-2022-23,Rashford-LIV,Home
5,LIV(H),Sancho,10,MID,3,2022-23,MUN,LIV,3-2022-23,Sancho-LIV,Home
6,BOU(H),Gündogan,10,MID,2,2022-23,MCI,BOU,2-2022-23,Gündogan-BOU,Home
7,BOU(H),De Bruyne,14,MID,2,2022-23,MCI,BOU,2-2022-23,De Bruyne-BOU,Home
8,BOU(H),Cancelo,11,DEF,2,2022-23,MCI,BOU,2-2022-23,Cancelo-BOU,Home
9,WHU(A),Haaland,13,FWD,1,2022-23,MCI,WHU,1-2022-23,Haaland-WHU,Away


In [141]:
hauls_df.groupby(['player']).size().sort_values(ascending = False).head(20)

player
Salah               51
Sterling            46
De Bruyne           44
Son                 41
Kane                40
Mané                37
Alexander-Arnold    33
Vardy               32
Mahrez              30
Hazard              26
Alonso              25
Firmino             24
Agüero              24
Martial             23
Willian             23
Sánchez             22
Aubameyang          20
David Silva         20
Doherty             19
Jesus               19
dtype: int64

In [142]:
hauls_df[hauls_df.player == 'Salah'].groupby(['fixture']).size().sort_values(ascending = False).head(25)

fixture
CRY(H)    4
ARS(H)    4
NOR(H)    3
WHU(A)    3
WAT(H)    3
BHA(A)    3
SOU(H)    3
BOU(A)    3
MCI(H)    2
SHU(H)    2
WAT(A)    2
HUD(H)    1
BHA(H)    1
BUR(H)    1
CAR(H)    1
CHE(H)    1
EVE(A)    1
EVE(H)    1
WOL(A)    1
LEE(H)    1
LEI(H)    1
WHU(H)    1
MUN(A)    1
MUN(H)    1
NEW(H)    1
dtype: int64

In [143]:
hauls_df.sort_values(by='points', ascending=False).head(50)

,fixture,player,points,position,round,year,club_name,aclub,GW,player-opp,Ground
656,WAT(H),Salah,29,MID,31,2017-18,LIV,WAT,31-2017-18,Salah-WAT,Home
1931,NOR(A),Antonio,26,MID,44,2020-21,WHU,NOR,44-2020-21,Antonio-NOR,Away
1496,NOR(A),Antonio,26,MID,44,2019-20,WHU,NOR,44-2019-20,Antonio-NOR,Away
2562,WAT(H),Jesus,24,FWD,34,2021-22,MCI,WAT,34-2021-22,Jesus-WAT,Home
1766,LEI(A),Alexander-Arnold,24,DEF,19,2019-20,LIV,LEI,19-2019-20,Alexander-Arnold-LEI,Away
2606,MUN(A),Salah,24,MID,9,2021-22,LIV,MUN,9-2021-22,Salah-MUN,Away
2201,LEI(A),Alexander-Arnold,24,DEF,19,2020-21,LIV,LEI,19-2020-21,Alexander-Arnold-LEI,Away
283,LEI(A),Kane,24,FWD,37,2016-17,TOT,LEI,37-2016-17,Kane-LEI,Away
2630,WOL(A),De Bruyne,24,MID,36,2021-22,MCI,WOL,36-2021-22,De Bruyne-WOL,Away
2284,NOR(H),Mount,24,MID,9,2021-22,CHE,NOR,9-2021-22,Mount-NOR,Home


In [144]:
#base url for FPL API endpoints
fpl_base_url = 'https://fantasy.premierleague.com/api/'

In [145]:
# get data from bootstrap-static endpoint
r= requests.get(fpl_base_url+'bootstrap-static/')
json = r.json()

In [146]:
#show the keys
json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [147]:
#- Contains summary of Gameweek data
events_df = pd.DataFrame(json['events']) 
# phases_df = pd.DataFrame(json['phases']) - Shows calendar months for game weeks
teams_df = pd.DataFrame(json['teams'])
players_df = pd.DataFrame(json['elements'])
element_stats_df = pd.DataFrame(json['element_stats'])
element_types_df = pd.DataFrame(json['element_types'])

In [148]:
events_df.head()

,id,name,deadline_time,average_entry_score,finished,data_checked,highest_scoring_entry,deadline_time_epoch,deadline_time_game_offset,highest_score,...,cup_leagues_created,h2h_ko_matches_created,chip_plays,most_selected,most_transferred_in,top_element,top_element_info,transfers_made,most_captained,most_vice_captained
0,1,Gameweek 1,2022-08-05T17:30:00Z,57,True,True,7309963.0,1659720600,0,119.0,...,False,False,"[{'chip_name': 'bboost', 'num_played': 170273}...",28.0,1.0,366.0,"{'id': 366, 'points': 15}",0,283.0,318.0
1,2,Gameweek 2,2022-08-13T10:00:00Z,57,True,True,3977474.0,1660384800,0,139.0,...,False,False,"[{'chip_name': 'bboost', 'num_played': 136909}...",28.0,318.0,28.0,"{'id': 28, 'points': 19}",12847501,318.0,283.0
2,3,Gameweek 3,2022-08-20T10:00:00Z,48,True,True,9105261.0,1660989600,0,118.0,...,False,False,"[{'chip_name': 'bboost', 'num_played': 118152}...",28.0,19.0,7.0,"{'id': 7, 'points': 16}",20175060,28.0,318.0
3,4,Gameweek 4,2022-08-27T10:00:00Z,51,False,False,9063280.0,1661594400,0,147.0,...,False,False,"[{'chip_name': 'bboost', 'num_played': 145393}...",28.0,225.0,279.0,"{'id': 279, 'points': 22}",16105042,283.0,28.0
4,5,Gameweek 5,2022-08-30T17:00:00Z,0,False,False,NaN,1661878800,0,NaN,...,False,False,[],NaN,NaN,NaN,None,0,NaN,NaN


In [149]:
players_df.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text
0,NaN,NaN,58822,0,0,-2,2,0,2,0.5,...,345,135,428,168,5.0,,3.0,,NaN,
1,NaN,NaN,84450,0,0,0,0,1,3,6.0,...,90,57,36,26,NaN,,NaN,,NaN,
2,NaN,NaN,153256,0,0,-1,1,0,3,1.0,...,373,142,319,145,NaN,,NaN,,NaN,
3,NaN,NaN,156074,0,0,-2,2,0,2,0.7,...,346,136,429,169,NaN,,NaN,,NaN,
4,75.0,NaN,167199,0,0,0,0,0,3,1.9,...,86,55,140,75,NaN,,4.0,,NaN,


In [150]:
#Define URL to get fixtures
#url = 'https://www.fantasynutmeg.com/api/fixtures/'
fix = fpl_base_url+'fixtures/'

#Request hosting server to fetch url

data = requests.get(fix)
fixtures_df = pd.DataFrame(data.json())
fixtures_df.head(50)

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id
0,2292810,1,True,True,1,2022-08-05T19:00:00Z,90,False,True,1,2.0,7,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",3,2,74911
1,2292813,1,True,True,4,2022-08-06T11:30:00Z,90,False,True,12,2.0,9,2.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",5,2,74914
2,2292811,1,True,True,2,2022-08-06T14:00:00Z,90,False,True,2,0.0,3,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,2,74912
3,2292814,1,True,True,5,2022-08-06T14:00:00Z,90,False,True,20,1.0,11,2.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,2,74915
4,2292816,1,True,True,7,2022-08-06T14:00:00Z,90,False,True,16,0.0,15,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,3,74917
5,2292817,1,True,True,8,2022-08-06T14:00:00Z,90,False,True,17,1.0,18,4.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,4,74918
6,2292812,1,True,True,3,2022-08-06T16:30:00Z,90,False,True,6,1.0,8,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",4,2,74913
7,2292815,1,True,True,6,2022-08-07T13:00:00Z,90,False,True,4,2.0,10,2.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,4,74916
8,2292818,1,True,True,9,2022-08-07T13:00:00Z,90,False,True,5,2.0,14,1.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,4,74919
9,2292819,1,True,True,10,2022-08-07T15:30:00Z,90,False,True,13,2.0,19,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",5,3,74920


In [151]:
fixtures_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   code                    380 non-null    int64  
 1   event                   380 non-null    int64  
 2   finished                380 non-null    bool   
 3   finished_provisional    380 non-null    bool   
 4   id                      380 non-null    int64  
 5   kickoff_time            380 non-null    object 
 6   minutes                 380 non-null    int64  
 7   provisional_start_time  380 non-null    bool   
 8   started                 380 non-null    bool   
 9   team_a                  380 non-null    int64  
 10  team_a_score            37 non-null     float64
 11  team_h                  380 non-null    int64  
 12  team_h_score            37 non-null     float64
 13  stats                   380 non-null    object 
 14  team_h_difficulty       380 non-null    in

In [152]:
players_df.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
       'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam',
       'news', 'news_added', 'now_cost', 'photo', 'points_per_game',
       'second_name', 'selected_by_percent', 'special', 'squad_number',
       'status', 'team', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'influence_rank', 'influence_rank_type', 'creativity_rank',
       'creativity_rank_type', 'threat_rank'

In [153]:
teams_now=dict(zip(teams_df.id, teams_df.short_name))
positions=dict(zip(element_types_df.id, element_types_df.singular_name_short))
players_df['club_name'] = players_df['team'].map(teams_now)
players_df['position'] = players_df['element_type'].map(positions)

In [154]:
players_df.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,club_name,position
0,NaN,NaN,58822,0,0,-2,2,0,2,0.5,...,428,168,5.0,,3.0,,NaN,,ARS,DEF
1,NaN,NaN,84450,0,0,0,0,1,3,6.0,...,36,26,NaN,,NaN,,NaN,,ARS,MID
2,NaN,NaN,153256,0,0,-1,1,0,3,1.0,...,319,145,NaN,,NaN,,NaN,,ARS,MID
3,NaN,NaN,156074,0,0,-2,2,0,2,0.7,...,429,169,NaN,,NaN,,NaN,,ARS,DEF
4,75.0,NaN,167199,0,0,0,0,0,3,1.9,...,140,75,NaN,,4.0,,NaN,,ARS,MID


In [155]:
players_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 601 entries, 0 to 600
Data columns (total 69 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   chance_of_playing_next_round          198 non-null    float64
 1   chance_of_playing_this_round          189 non-null    float64
 2   code                                  601 non-null    int64  
 3   cost_change_event                     601 non-null    int64  
 4   cost_change_event_fall                601 non-null    int64  
 5   cost_change_start                     601 non-null    int64  
 6   cost_change_start_fall                601 non-null    int64  
 7   dreamteam_count                       601 non-null    int64  
 8   element_type                          601 non-null    int64  
 9   ep_next                               601 non-null    object 
 10  ep_this                               601 non-null    object 
 11  event_points       

In [156]:
players_df.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
       'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam',
       'news', 'news_added', 'now_cost', 'photo', 'points_per_game',
       'second_name', 'selected_by_percent', 'special', 'squad_number',
       'status', 'team', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'influence_rank', 'influence_rank_type', 'creativity_rank',
       'creativity_rank_type', 'threat_rank'

In [157]:
players_df[['first_name', 'second_name', 'club_name', 'position', 'id', 'form', 'ep_next', 'ep_this']].head(10)

,first_name,second_name,club_name,position,id,form,ep_next,ep_this
0,Cédric,Alves Soares,ARS,DEF,1,0.0,0.5,1.0
1,Granit,Xhaka,ARS,MID,3,5.5,6.0,6.5
2,Mohamed,Elneny,ARS,MID,4,0.5,1.0,1.5
3,Rob,Holding,ARS,DEF,5,0.2,0.7,1.2
4,Thomas,Partey,ARS,MID,6,2.0,1.9,3.0
5,Martin,Ødegaard,ARS,MID,7,7.8,8.3,8.8
6,Kieran,Tierney,ARS,DEF,8,1.2,1.7,2.2
7,Nicolas,Pépé,ARS,MID,9,0.0,0.0,0.0
8,Benjamin,White,ARS,DEF,10,3.5,4.0,4.5
9,Eddie,Nketiah,ARS,FWD,11,1.0,1.5,2.0


In [158]:
#Data Cleaning
players_df.chance_of_playing_next_round = players_df.chance_of_playing_next_round.fillna(100.0)
players_df.chance_of_playing_this_round = players_df.chance_of_playing_this_round.fillna(100.0)
players_df.drop('id', axis=1, inplace=True)
fixtures_df.team_a_score = fixtures_df.team_a_score.fillna(0)
fixtures_df.team_h_score = fixtures_df.team_h_score.fillna(0)
fixtures_df.drop(['stats', 'id'], axis=1, inplace=True)
fixtures_df['kickoff_time'] = fixtures_df['kickoff_time'].str[:-10]
fixtures_df['kickoff_time'] = pd.to_datetime(fixtures_df['kickoff_time'])

In [159]:
fixtures_df.columns

Index(['code', 'event', 'finished', 'finished_provisional', 'kickoff_time',
       'minutes', 'provisional_start_time', 'started', 'team_a',
       'team_a_score', 'team_h', 'team_h_score', 'team_h_difficulty',
       'team_a_difficulty', 'pulse_id'],
      dtype='object')

In [160]:
fixtures_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   code                    380 non-null    int64         
 1   event                   380 non-null    int64         
 2   finished                380 non-null    bool          
 3   finished_provisional    380 non-null    bool          
 4   kickoff_time            380 non-null    datetime64[ns]
 5   minutes                 380 non-null    int64         
 6   provisional_start_time  380 non-null    bool          
 7   started                 380 non-null    bool          
 8   team_a                  380 non-null    int64         
 9   team_a_score            380 non-null    float64       
 10  team_h                  380 non-null    int64         
 11  team_h_score            380 non-null    float64       
 12  team_h_difficulty       380 non-null    int64     

In [161]:
fixtures_df.head()

,code,event,finished,finished_provisional,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,team_h_difficulty,team_a_difficulty,pulse_id
0,2292810,1,True,True,2022-08-05,90,False,True,1,2.0,7,0.0,3,2,74911
1,2292813,1,True,True,2022-08-06,90,False,True,12,2.0,9,2.0,5,2,74914
2,2292811,1,True,True,2022-08-06,90,False,True,2,0.0,3,2.0,2,2,74912
3,2292814,1,True,True,2022-08-06,90,False,True,20,1.0,11,2.0,2,2,74915
4,2292816,1,True,True,2022-08-06,90,False,True,16,0.0,15,2.0,2,3,74917


In [162]:
#Create Next Game week fixtures
#GW3fixtures_df = fixtures_df.loc[(fixtures_df['kickoff_time'] >= "2022-08-27") & (fixtures_df['kickoff_time'] <= "2022-08-28")].copy()
#GW3fixtures_df.head()

In [163]:
#Create Next Game week fixtures
GWfixtures_df = fixtures_df.loc[(fixtures_df['event'] == 4)].copy()
GWfixtures_df.head()

,code,event,finished,finished_provisional,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,team_h_difficulty,team_a_difficulty,pulse_id
30,2292848,4,True,True,2022-08-27,90,False,True,14,1.0,17,0.0,3,2,74949
31,2292842,4,True,True,2022-08-27,90,False,True,8,1.0,4,1.0,2,2,74943
32,2292843,4,True,True,2022-08-27,90,False,True,11,0.0,5,1.0,2,2,74944
33,2292844,4,True,True,2022-08-27,90,False,True,10,1.0,6,2.0,3,4,74945
34,2292845,4,True,True,2022-08-27,90,False,True,3,0.0,12,9.0,2,5,74946


In [164]:
GWfixtures_df.shape

(10, 15)

In [165]:
#Mapping away and home team and respective strengths
home_strength= dict(zip(teams_df.id, teams_df.strength_overall_home))
away_strength= dict(zip(teams_df.id, teams_df.strength_overall_away))
GWfixtures_df['away_team'] = GWfixtures_df['team_a'].map(teams_now)
GWfixtures_df['home_team'] = GWfixtures_df['team_h'].map(teams_now)
GWfixtures_df['away_team_strength'] = GWfixtures_df['team_a'].map(away_strength)
GWfixtures_df['home_team_strength'] = GWfixtures_df['team_h'].map(home_strength)
GWfixtures_df.head()

,code,event,finished,finished_provisional,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,team_h_difficulty,team_a_difficulty,pulse_id,away_team,home_team,away_team_strength,home_team_strength
30,2292848,4,True,True,2022-08-27,90,False,True,14,1.0,17,0.0,3,2,74949,MUN,SOU,1250,1060
31,2292842,4,True,True,2022-08-27,90,False,True,8,1.0,4,1.0,2,2,74943,EVE,BRE,1100,1090
32,2292843,4,True,True,2022-08-27,90,False,True,11,0.0,5,1.0,2,2,74944,LEE,BHA,1070,1100
33,2292844,4,True,True,2022-08-27,90,False,True,10,1.0,6,2.0,3,4,74945,LEI,CHE,1240,1270
34,2292845,4,True,True,2022-08-27,90,False,True,3,0.0,12,9.0,2,5,74946,BOU,LIV,1050,1350


In [166]:
GWfixtures_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 30 to 39
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   code                    10 non-null     int64         
 1   event                   10 non-null     int64         
 2   finished                10 non-null     bool          
 3   finished_provisional    10 non-null     bool          
 4   kickoff_time            10 non-null     datetime64[ns]
 5   minutes                 10 non-null     int64         
 6   provisional_start_time  10 non-null     bool          
 7   started                 10 non-null     bool          
 8   team_a                  10 non-null     int64         
 9   team_a_score            10 non-null     float64       
 10  team_h                  10 non-null     int64         
 11  team_h_score            10 non-null     float64       
 12  team_h_difficulty       10 non-null     int64      

In [167]:
players_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 601 entries, 0 to 600
Data columns (total 68 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   chance_of_playing_next_round          601 non-null    float64
 1   chance_of_playing_this_round          601 non-null    float64
 2   code                                  601 non-null    int64  
 3   cost_change_event                     601 non-null    int64  
 4   cost_change_event_fall                601 non-null    int64  
 5   cost_change_start                     601 non-null    int64  
 6   cost_change_start_fall                601 non-null    int64  
 7   dreamteam_count                       601 non-null    int64  
 8   element_type                          601 non-null    int64  
 9   ep_next                               601 non-null    object 
 10  ep_this                               601 non-null    object 
 11  event_points       

In [168]:
players_df.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,club_name,position
0,100.0,100.0,58822,0,0,-2,2,0,2,0.5,...,428,168,5.0,,3.0,,NaN,,ARS,DEF
1,100.0,100.0,84450,0,0,0,0,1,3,6.0,...,36,26,NaN,,NaN,,NaN,,ARS,MID
2,100.0,100.0,153256,0,0,-1,1,0,3,1.0,...,319,145,NaN,,NaN,,NaN,,ARS,MID
3,100.0,100.0,156074,0,0,-2,2,0,2,0.7,...,429,169,NaN,,NaN,,NaN,,ARS,DEF
4,75.0,100.0,167199,0,0,0,0,0,3,1.9,...,140,75,NaN,,4.0,,NaN,,ARS,MID


In [169]:
players_df[['first_name', 'second_name', 'club_name', 'position', 'form', 'team']].head()

,first_name,second_name,club_name,position,form,team
0,Cédric,Alves Soares,ARS,DEF,0.0,1
1,Granit,Xhaka,ARS,MID,5.5,1
2,Mohamed,Elneny,ARS,MID,0.5,1
3,Rob,Holding,ARS,DEF,0.2,1
4,Thomas,Partey,ARS,MID,2.0,1


In [170]:
GWfixtures_df.head()

,code,event,finished,finished_provisional,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,team_h_difficulty,team_a_difficulty,pulse_id,away_team,home_team,away_team_strength,home_team_strength
30,2292848,4,True,True,2022-08-27,90,False,True,14,1.0,17,0.0,3,2,74949,MUN,SOU,1250,1060
31,2292842,4,True,True,2022-08-27,90,False,True,8,1.0,4,1.0,2,2,74943,EVE,BRE,1100,1090
32,2292843,4,True,True,2022-08-27,90,False,True,11,0.0,5,1.0,2,2,74944,LEE,BHA,1070,1100
33,2292844,4,True,True,2022-08-27,90,False,True,10,1.0,6,2.0,3,4,74945,LEI,CHE,1240,1270
34,2292845,4,True,True,2022-08-27,90,False,True,3,0.0,12,9.0,2,5,74946,BOU,LIV,1050,1350


In [171]:
gw_away_players = pd.merge(players_df, GWfixtures_df, how="inner", left_on=["team"], right_on=["team_a"])
gw_away_players['player_opp'] = gw_away_players['web_name'].map(str) + '-' + gw_away_players['home_team'].map(str)
gw_home_players = pd.merge(players_df, GWfixtures_df, how="inner", left_on=["team"], right_on=["team_h"])
gw_home_players['player_opp'] = gw_home_players['web_name'].map(str) + '-' + gw_home_players['away_team'].map(str)
gw_away_players['diff'] = gw_away_players['away_team_strength'] - gw_away_players['home_team_strength']
gw_home_players['diff'] = gw_home_players['home_team_strength'] - gw_home_players['away_team_strength']
players_df = gw_away_players.append(gw_home_players)
players_df['player_club'] = players_df['web_name'].map(str) + '-' + players_df['club_name'].map(str)

In [172]:
players_df[['first_name', 'second_name', 'web_name', 'club_name', 'position', 'form', 'diff', 'away_team', 'home_team', 'player_club', 'player_opp' ]].head(100)

,first_name,second_name,web_name,club_name,position,form,diff,away_team,home_team,player_club,player_opp
0,Adam,Smith,Smith,BOU,DEF,0.5,-300,BOU,LIV,Smith-BOU,Smith-LIV
1,Junior,Stanislas,Stanislas,BOU,MID,0.5,-300,BOU,LIV,Stanislas-BOU,Stanislas-LIV
2,Ryan,Fredericks,Fredericks,BOU,DEF,0.0,-300,BOU,LIV,Fredericks-BOU,Fredericks-LIV
3,David,Brooks,Brooks,BOU,MID,0.0,-300,BOU,LIV,Brooks-BOU,Brooks-LIV
4,Jamal,Lowe,Lowe,BOU,MID,0.2,-300,BOU,LIV,Lowe-BOU,Lowe-LIV
...,...,...,...,...,...,...,...,...,...,...,...
95,Neeskens,Kebano,Kebano,FUL,MID,1.8,-140,FUL,ARS,Kebano-FUL,Kebano-ARS
96,Bobby,De Cordova-Reid,De Cordova-Reid,FUL,MID,3.2,-140,FUL,ARS,De Cordova-Reid-FUL,De Cordova-Reid-ARS
97,Joe,Bryan,Bryan,FUL,DEF,0.0,-140,FUL,ARS,Bryan-FUL,Bryan-ARS
98,Paulo,Gazzaniga Farias,Gazzaniga,FUL,GKP,0.0,-140,FUL,ARS,Gazzaniga-FUL,Gazzaniga-ARS


In [173]:
#My team
my_team = ['Ederson-MCI', 'Alexander-Arnold-LIV', 'Salah-LIV', 'Walker-MCI', 'Haaland-MCI', 'James-CHE', 'Cash-AVL', 'Archer-AVL', 'Rodrigo-LEE', 'Saka-ARS', 'Martinelli-ARS', 'Jesus-ARS', 'Meslier-LEE', 'Milivojevic-CRY', 'Dunk-BHA']
my_players_df = players_df.loc[players_df.player_club.isin(my_team)].copy()
my_players_df.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code_x,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,team_h_difficulty,team_a_difficulty,pulse_id,away_team,home_team,away_team_strength,home_team_strength,player_opp,diff,player_club
34,100.0,100.0,66975,0,0,0,0,0,3,0.8,...,2,5,74947,CRY,MCI,1100,1350,Milivojevic-MCI,-250,Milivojevic-CRY
152,100.0,100.0,80954,0,0,4,-4,2,3,8.7,...,2,2,74944,LEE,BHA,1070,1100,Rodrigo-BHA,-30,Rodrigo-LEE
165,100.0,100.0,437495,0,0,0,0,0,1,3.0,...,2,2,74944,LEE,BHA,1070,1100,Meslier-BHA,-30,Meslier-LEE
11,100.0,100.0,223340,0,0,-1,1,0,3,4.5,...,2,4,74941,FUL,ARS,1060,1200,Saka-FUL,140,Saka-ARS
17,100.0,100.0,444145,0,0,4,-4,0,3,6.5,...,2,4,74941,FUL,ARS,1060,1200,Martinelli-FUL,140,Martinelli-ARS


In [174]:
bank = 2
my_team = ['Ederson-MCI', 'Alexander-Arnold-LIV', 'Salah-LIV', 'Walker-MCI', 'Haaland-MCI', 'James-CHE', 'Cash-AVL', 'Archer-AVL', 'Rodrigo-LEE', 'Saka-ARS', 'Martinelli-ARS', 'Jesus-ARS', 'Meslier-LEE', 'Milivojevic-CRY', 'Dunk-BHA']
potential_players = players_df.loc[~players_df.player_club.isin(my_team)].copy()
potential_players.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code_x,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,team_h_difficulty,team_a_difficulty,pulse_id,away_team,home_team,away_team_strength,home_team_strength,player_opp,diff,player_club
0,100.0,100.0,54469,0,0,-1,1,0,2,0.0,...,2,5,74946,BOU,LIV,1050,1350,Smith-LIV,-300,Smith-BOU
1,0.0,0.0,56872,0,0,-1,1,0,3,0.0,...,2,5,74946,BOU,LIV,1050,1350,Stanislas-LIV,-300,Stanislas-BOU
2,0.0,0.0,81012,0,0,-1,1,0,2,0.0,...,2,5,74946,BOU,LIV,1050,1350,Fredericks-LIV,-300,Fredericks-BOU
3,0.0,0.0,111317,0,0,-1,1,0,3,0.0,...,2,5,74946,BOU,LIV,1050,1350,Brooks-LIV,-300,Brooks-BOU
4,75.0,75.0,128309,0,0,-1,1,0,3,-0.2,...,2,5,74946,BOU,LIV,1050,1350,Lowe-LIV,-300,Lowe-BOU


In [175]:
potential_players.shape

(586, 90)

In [176]:
my_players_df[['web_name', 'club_name', 'position', 'form', 'now_cost', 'away_team', 'home_team', 'chance_of_playing_this_round']].head(20)

,web_name,club_name,position,form,now_cost,away_team,home_team,chance_of_playing_this_round
34,Milivojevic,CRY,MID,0.8,45,CRY,MCI,100.0
152,Rodrigo,LEE,MID,9.2,64,LEE,BHA,100.0
165,Meslier,LEE,GKP,3.5,45,LEE,BHA,100.0
11,Saka,ARS,MID,4.0,79,FUL,ARS,100.0
17,Martinelli,ARS,MID,6.0,64,FUL,ARS,100.0
26,Jesus,ARS,FWD,6.5,82,FUL,ARS,100.0
42,Cash,AVL,DEF,1.0,48,WHU,AVL,100.0
52,Archer,AVL,FWD,0.2,45,WHU,AVL,100.0
93,Dunk,BHA,DEF,5.5,45,LEE,BHA,100.0
139,James,CHE,DEF,5.5,61,LEI,CHE,100.0


In [177]:
#Deciding Players out
def calc_out_weights(players):
    players['weight'] = 100
    players['weight'] -= players['diff']/3
    players['weight'] -= players['form'].astype("float")*10
    players['weight']+= (100 -players['chance_of_playing_this_round'].astype("float"))*0.2
    players.loc[players['element_type'] ==1, 'weight'] -=10
    players.loc[players['weight'] <0, 'weight'] =0
    return players.sample(1, weights=players.weight)

In [178]:
player_out= calc_out_weights(my_players_df)
player_out[['web_name', 'element_type']]

,web_name,element_type
42,Cash,2


In [179]:
my_players_df[['web_name', 'club_name', 'team', 'position', 'chance_of_playing_this_round', 'weight', 'now_cost', 'form', 'diff', 'away_team', 'home_team']].sort_values(by='weight', ascending=False).head(20)

,web_name,club_name,team,position,chance_of_playing_this_round,weight,now_cost,form,diff,away_team,home_team
34,Milivojevic,CRY,7,MID,100.0,175.333333,45,0.8,-250,CRY,MCI
52,Archer,AVL,2,FWD,100.0,124.666667,45,0.2,-80,WHU,AVL
42,Cash,AVL,2,DEF,100.0,116.666667,48,1.0,-80,WHU,AVL
165,Meslier,LEE,11,GKP,100.0,65.000000,45,3.5,-30,LEE,BHA
93,Dunk,BHA,5,DEF,100.0,35.000000,45,5.5,30,LEE,BHA
139,James,CHE,6,DEF,100.0,35.000000,61,5.5,30,LEI,CHE
152,Rodrigo,LEE,11,MID,100.0,18.000000,64,9.2,-30,LEE,BHA
11,Saka,ARS,1,MID,100.0,13.333333,79,4.0,140,FUL,ARS
17,Martinelli,ARS,1,MID,100.0,0.000000,64,6.0,140,FUL,ARS
26,Jesus,ARS,1,FWD,100.0,0.000000,82,6.5,140,FUL,ARS


In [180]:
##Deciding players In
position = player_out.element_type.iat[0]
#position = 2
out_cost = player_out.now_cost.iat[0]
#out_cost = 49
budget = bank + out_cost
dups_team = my_players_df.pivot_table(index=['club_name'], aggfunc='size')
invalid_teams = dups_team.loc[dups_team==3].index.tolist()
invalid_teams

['ARS', 'MCI']

In [181]:
potential_players=potential_players.loc[~potential_players.club_name.isin(invalid_teams)]
potential_players=potential_players.loc[potential_players.element_type==position]
potential_players = potential_players.loc[potential_players.now_cost<=budget]
potential_players[['web_name', 'element_type','club_name', 'position', 'chance_of_playing_this_round', 'now_cost', 'form', 'diff', 'away_team', 'home_team']].head(11)

,web_name,element_type,club_name,position,chance_of_playing_this_round,now_cost,form,diff,away_team,home_team
0,Smith,2,BOU,DEF,100.0,44,0.5,-300,BOU,LIV
2,Fredericks,2,BOU,DEF,0.0,44,0.0,-300,BOU,LIV
8,Stacey,2,BOU,DEF,100.0,44,-0.2,-300,BOU,LIV
14,Mepham,2,BOU,DEF,100.0,45,0.8,-300,BOU,LIV
16,Kelly,2,BOU,DEF,100.0,45,3.2,-300,BOU,LIV
19,Zemura,2,BOU,DEF,100.0,44,1.0,-300,BOU,LIV
23,Hill,2,BOU,DEF,100.0,45,0.0,-300,BOU,LIV
25,Senesi,2,BOU,DEF,100.0,45,-0.3,-300,BOU,LIV
27,Bevan,2,BOU,DEF,100.0,40,1.0,-300,BOU,LIV
29,Tomkins,2,CRY,DEF,0.0,40,0.0,-250,CRY,MCI


In [182]:
potential_players.shape

(181, 90)

In [183]:
#Confirm Invalid Teams are not included
potential_players.groupby(['now_cost']).size().sort_values(ascending = False).head(20)

now_cost
45    65
44    43
40    41
50    11
49     8
48     6
47     2
43     2
46     1
41     1
39     1
dtype: int64

In [184]:
def calc_in_players_weights(players):
    players['weight'] = 1
    players['weight'] += players['diff']/3
    players['weight'] += players['form'].astype("float")*10
    players['weight'] -= (100 - players['chance_of_playing_this_round'].astype("float"))*0.2
    players.loc[players['weight'] <0, 'weight'] =0
    return players

In [185]:
player_in = calc_in_players_weights(potential_players)
player_in[['web_name', 'club_name', 'element_type', 'chance_of_playing_this_round', 'weight', 'now_cost', 'form', 'diff', 'away_team', 'home_team']]

,web_name,club_name,element_type,chance_of_playing_this_round,weight,now_cost,form,diff,away_team,home_team
0,Smith,BOU,2,100.0,0.0,44,0.5,-300,BOU,LIV
2,Fredericks,BOU,2,0.0,0.0,44,0.0,-300,BOU,LIV
8,Stacey,BOU,2,100.0,0.0,44,-0.2,-300,BOU,LIV
14,Mepham,BOU,2,100.0,0.0,45,0.8,-300,BOU,LIV
16,Kelly,BOU,2,100.0,0.0,45,3.2,-300,BOU,LIV
...,...,...,...,...,...,...,...,...,...,...
284,Aït-Nouri,WOL,2,100.0,0.0,45,1.5,-90,NEW,WOL
286,Toti,WOL,2,100.0,0.0,40,0.0,-90,NEW,WOL
290,Mosquera,WOL,2,100.0,0.0,40,0.0,-90,NEW,WOL
292,Collins,WOL,2,100.0,0.0,45,2.5,-90,NEW,WOL


In [186]:
potential_players = player_in.loc[(player_in.weight>0) & (player_in.chance_of_playing_this_round==100)] 
potential_players[['web_name', 'club_name', 'position', 'chance_of_playing_this_round', 'weight', 'now_cost', 'form', 'diff', 'away_team', 'home_team', 'player_opp']].sort_values(by='form', ascending=False).head(40)

,web_name,club_name,position,chance_of_playing_this_round,weight,now_cost,form,diff,away_team,home_team,player_opp
96,Veltman,BHA,DEF,100.0,71.000000,45,6.0,30,LEE,BHA,Veltman-LEE
227,Schär,NEW,DEF,100.0,81.000000,47,5.0,90,NEW,WOL,Schär-WOL
95,Webster,BHA,DEF,100.0,59.000000,45,4.8,30,LEE,BHA,Webster-LEE
84,Mee,BRE,DEF,100.0,37.666667,46,4.0,-10,EVE,BRE,Mee-EVE
261,Emerson Royal,TOT,DEF,100.0,117.666667,50,4.0,230,TOT,NFO,Emerson Royal-NFO
246,Dier,TOT,DEF,100.0,115.666667,50,3.8,230,TOT,NFO,Dier-NFO
162,Struijk,LEE,DEF,100.0,29.000000,45,3.8,-30,LEE,BHA,Struijk-BHA
219,Burn,NEW,DEF,100.0,66.000000,45,3.5,90,NEW,WOL,Burn-WOL
116,Estupiñán,BHA,DEF,100.0,46.000000,45,3.5,30,LEE,BHA,Estupiñán-LEE
155,Llorente,LEE,DEF,100.0,23.000000,45,3.2,-30,LEE,BHA,Llorente-BHA


In [187]:
potential_players.shape

(74, 91)

In [188]:
#Player by random selection
potential_player_alg = potential_players.sample(1, weights=potential_players.weight)
potential_player_alg[['web_name', 'club_name', 'position', 'chance_of_playing_this_round', 'weight', 'now_cost', 'form', 'diff', 'away_team', 'home_team', 'player_opp']].head()

,web_name,club_name,position,chance_of_playing_this_round,weight,now_cost,form,diff,away_team,home_team,player_opp
183,Varane,MUN,DEF,100.0,84.333333,48,2.0,190,MUN,SOU,Varane-SOU


In [189]:
#Checking Players with Historical Haul
potential_players2 = pd.merge(potential_players, hauls_df, how="inner", left_on=["player_opp"], right_on=["player-opp"])

In [190]:
potential_players2[['web_name', 'player_opp', 'club_name_y', 'year']].head()

,web_name,player_opp,club_name_y,year
0,Maguire,Maguire-SOU,LEI,2018-19
1,Shaw,Shaw-SOU,MUN,2016-17
2,Lascelles,Lascelles-WOL,NEW,2019-20
3,Lascelles,Lascelles-WOL,NEW,2020-21


In [191]:
def calc_starters_weights(players):
    players['weight'] = 1
    players['weight'] += players['diff']/2
    players['weight'] += players['form'].astype("float")*5
    players['weight'] -= (100 - players['chance_of_playing_this_round'].astype("float"))*0.2
    players.loc[players['weight'] <0, 'weight'] =0
    return players

In [192]:
starters = calc_starters_weights(my_players_df)
starters[['web_name', 'club_name', 'team', 'position', 'chance_of_playing_this_round', 'weight', 'now_cost', 'form', 'diff', 'away_team', 'home_team']].sort_values(by='weight', ascending=False).head(15)

,web_name,club_name,team,position,chance_of_playing_this_round,weight,now_cost,form,diff,away_team,home_team
156,Salah,LIV,12,MID,100.0,182.0,130,6.2,300,BOU,LIV
158,Alexander-Arnold,LIV,12,DEF,100.0,177.0,75,5.2,300,BOU,LIV
197,Haaland,MCI,13,FWD,100.0,177.0,117,10.2,250,CRY,MCI
187,Ederson,MCI,13,GKP,100.0,145.0,55,3.8,250,CRY,MCI
180,Walker,MCI,13,DEF,100.0,143.5,51,3.5,250,CRY,MCI
26,Jesus,ARS,1,FWD,100.0,103.5,82,6.5,140,FUL,ARS
17,Martinelli,ARS,1,MID,100.0,101.0,64,6.0,140,FUL,ARS
11,Saka,ARS,1,MID,100.0,91.0,79,4.0,140,FUL,ARS
93,Dunk,BHA,5,DEF,100.0,43.5,45,5.5,30,LEE,BHA
139,James,CHE,6,DEF,100.0,43.5,61,5.5,30,LEI,CHE
